## Integration tests of federated data in time series 

In [1]:
import sys
import pandas as pd
import numpy as np
sys.path.append("/data/Beatriz/Doctorado GR/ADL_platform/S-ADL")


In [2]:
from flexanomalies.datasets.preprocessing_utils import scaling
from flexanomalies.utils.load_data import split_data
from SADL.federated_data.algorithms import flexanomalies
from SADL.federated_data import federated_time_series_utils


2025-02-03 20:26:04.667032: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-03 20:26:04.668384: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-03 20:26:04.671614: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-03 20:26:04.680367: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738610764.695427   14624 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738610764.69

##### Example of time series data set for testing

In [3]:
file_path = "/data/Beatriz/Doctorado GR/ADL_platform/S-ADL/datasets/data/corrected.gz"
split_test = 0.3

df = pd.read_csv(file_path, header=None)
print(df)
# process labels
df.loc[df[41] != "normal.", 41] = 1
df.loc[df[41] == "normal.", 41] = 0
labels = df[41]
df = df.drop([41], axis=1)

features_to_encode = [1, 2, 3]
df = df.drop(features_to_encode, axis=1)
indim = df.shape[1]


        0    1        2   3    4    5   6   7   8   9   ...   32   33    34  \
0        0  udp  private  SF  105  146   0   0   0   0  ...  254  1.0  0.01   
1        0  udp  private  SF  105  146   0   0   0   0  ...  254  1.0  0.01   
2        0  udp  private  SF  105  146   0   0   0   0  ...  254  1.0  0.01   
3        0  udp  private  SF  105  146   0   0   0   0  ...  254  1.0  0.01   
4        0  udp  private  SF  105  146   0   0   0   0  ...  254  1.0  0.01   
...     ..  ...      ...  ..  ...  ...  ..  ..  ..  ..  ...  ...  ...   ...   
311024   0  udp  private  SF  105  147   0   0   0   0  ...  255  1.0  0.00   
311025   0  udp  private  SF  105  147   0   0   0   0  ...  255  1.0  0.00   
311026   0  udp  private  SF  105  147   0   0   0   0  ...  255  1.0  0.00   
311027   0  udp  private  SF  105  147   0   0   0   0  ...  255  1.0  0.00   
311028   0  udp  private  SF  105  147   0   0   0   0  ...  255  1.0  0.00   

          35   36   37   38   39   40              

In [4]:
X = scaling(np.array(df.iloc[:, :].astype(float)))
y = np.array(labels)
X_train, X_test, y_train, y_test = split_data(X, y, split_size= split_test)


#### Autoencoder Model for anomaly detection in time series 

In [5]:
kwargs = {
    "algorithm_": "autoencoder",
    "contamination":0.1,
    "label_parser": None,
    "epochs": 3,
    "input_dim": indim,
    "batch_size": 16,
    "neurons": [16, 8, 16],
    "hidden_act": ["relu", "relu", "relu"],
    "preprocess":False,
    "w_size": 10,
    "n_pred": 2,
     "n_clients":3,
     "n_rounds":3,
     }



modelAE = flexanomalies.FlexAnomalyDetection(**kwargs)
print(modelAE.get_params())


Federated Params:{'n_clients': 3, 'n_rounds': 3} 
 Model Params:{'algorithm_': 'autoencoder', 'contamination': 0.1, 'label_parser': None, 'epochs': 3, 'input_dim': 38, 'batch_size': 16, 'neurons': [16, 8, 16], 'hidden_act': ['relu', 'relu', 'relu'], 'preprocess': False, 'w_size': 10, 'n_pred': 2}
{'algorithm_': 'autoencoder', 'contamination': 0.1, 'label_parser': None, 'epochs': 3, 'input_dim': 38, 'batch_size': 16, 'neurons': [16, 8, 16], 'hidden_act': ['relu', 'relu', 'relu'], 'preprocess': False, 'w_size': 10, 'n_pred': 2}
odict_values([<Parameter "self">, <Parameter "input_dim">, <Parameter "neurons">, <Parameter "model_path=''">, <Parameter "callbacks=[]">, <Parameter "hidden_act='relu'">, <Parameter "output_act='linear'">, <Parameter "loss='mse'">, <Parameter "validation_size=0.2">, <Parameter "batch_size=32">, <Parameter "epochs=1">, <Parameter "optimizer='adam'">, <Parameter "contamination=0.1">, <Parameter "w_size=None">, <Parameter "n_pred=1">, <Parameter "preprocess=True">])

/data/Beatriz/Doctorado GR/ADL_platform/S-ADL/venv/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-02-03 19:10:56.573954: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [6]:
processor =   federated_time_series_utils.TimeSeriesProcessor(window_size= kwargs["w_size"], step_size=1, future_prediction=False)
X_train_windows, y_train_windows, X_test_windows, y_test_windows = processor.process_train_test(X_train, y_train, X_test, y_test)
print("X_train shape:", X_train_windows.shape)
print("y_train shape:", y_train_windows.shape)
print("X_test shape:", X_test_windows.shape)
print("y_test shape:", y_test_windows.shape)

X_train shape: (217711, 10, 38)
y_train shape: (217711, 10)
X_test shape: (93300, 10, 38)
y_test shape: (93300, 10)


In [7]:
modelAE.fit(X_train_windows,y_train_windows)



Running round: 0

Training model at client.


/data/Beatriz/Doctorado GR/ADL_platform/S-ADL/venv/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 18 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Epoch 1/3
3266/3266 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - loss: 0.5147 - val_loss: 0.1838
Epoch 2/3
3266/3266 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 0.1986 - val_loss: 0.1405
Epoch 3/3
3266/3266 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - loss: 0.1327 - val_loss: 0.1149
Training model at client.
Epoch 1/3
3266/3266 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - loss: 0.4707 - val_loss: 0.3144
Epoch 2/3
3266/3266 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - loss: 0.1537 - val_loss: 0.2359
Epoch 3/3
3266/3266 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - loss: 0.1303 - val_loss: 0.1463
Training model at client.
Epoch 1/3
3266/3266 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - loss: 0.5684 - val_loss: 0.2861
Epoch 2/3
3266/3266 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 0.2203 - val_loss: 0.2019
Epoch 3/3
3266/3266 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - loss: 0.1623 - val_loss: 0.1749

Running round: 1

Training model at client.
Epoch 1/3
3266/3266 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - loss: 0.2798 - val_loss: 0.1605
Epoch 2/3
3266/3266 ━━━━━━━━━━━━━

In [8]:
scores_pred = modelAE.decision_function(X_train_windows)
print("Scores",scores_pred)
modelAE.predict(X_train_windows)

6804/6804 ━━━━━━━━━━━━━━━━━━━━ 5s 671us/step
Scores [[0.99328867 0.57382054 3.00803215 ... 0.22773626 0.91034639 1.01015595]
 [1.00253486 0.5743122  3.0081699  ... 0.24410959 0.90578856 1.00099864]
 [0.34366815 0.14053562 0.76891684 ... 0.20282594 0.20023528 0.26582654]
 ...
 [0.95825015 0.23355521 1.88239588 ... 0.34331429 0.42264287 0.42528602]
 [0.95625395 0.22184052 1.89434226 ... 0.33740468 0.42415495 0.42624949]
 [0.95702854 0.2213175  1.89433961 ... 0.33729577 0.41228807 0.40776362]]
Inspecting model's attributes:
contamination: 0.1
input_dim: 38
neurons: [16, 8, 16]
model_path: 
w_size: 10
n_pred: 2
callbacks: [<keras.src.callbacks.early_stopping.EarlyStopping object at 0x7c14b65cded0>]
hidden_act: ['relu', 'relu', 'relu']
output_act: linear
loss: mse
validation_size: 0.2
batch_size: 16
epochs: 3
optimizer: adam
preprocess: False
scaler: StandardScaler()
model: <Sequential name=sequential_1, built=True>
algorithm_: autoencoder
label_parser: None
6804/6804 ━━━━━━━━━━━━━━━━━━━━ 5

array([[[ 3.1035960e-02, -1.1627361e-02, -3.0039489e-02, ...,
         -2.8931910e-01,  2.5805795e+00,  2.6128571e+00],
        [ 9.0102434e-01, -5.6949693e-01,  3.0293453e+00, ...,
         -3.9226642e-01, -1.3121579e+00, -1.3963451e+00],
        [-2.4289787e-02,  3.2790005e-03, -4.2930871e-02, ...,
          3.9826097e+00, -4.6081954e-01, -4.4598505e-01],
        ...,
        [-1.8526703e-01, -3.1662643e-02,  1.5330273e-01, ...,
         -2.9990008e-01, -3.9725322e-01, -4.0651619e-01],
        [-1.9917542e-01,  1.8193051e-02, -7.9798371e-02, ...,
         -3.4760281e-01, -4.1070485e-01, -3.9620018e-01],
        [-8.6194307e-02, -1.0096440e-01, -2.9848218e-02, ...,
         -2.5389358e-01, -4.3009770e-01, -4.4160140e-01]],

       [[ 9.0102434e-01, -5.6949693e-01,  3.0293453e+00, ...,
         -3.9226642e-01, -1.3121579e+00, -1.3963451e+00],
        [-2.4289787e-02,  3.2790005e-03, -4.2930871e-02, ...,
          3.9826097e+00, -4.6081954e-01, -4.4598505e-01],
        [-1.9856858e-01, 

In [10]:
modelAE.model.d_scores_ ==scores_pred
modelAE.model.labels_


array([1., 1., 1., ..., 0., 0., 0.])

In [15]:
modelAE.evaluate(X_test_windows, X_test_windows, label_test = y_test_windows.flatten().astype("int"))

[1 1 1 ... 1 0 1]
2916/2916 ━━━━━━━━━━━━━━━━━━━━ 2s 688us/step
Acc: 62.881% 

Precision: 0.859 

F1score: 0.737 

Recall: 0.646 

AUC_ROC: 0.602 



#### CNN_LSTM model for anomaly forcasting  in time series 

In [ ]:
kwargs = {
    "algorithm_": "deepCNN_LSTM",
    "contamination":0.1,
    "label_parser": None,
    "epochs": 3,
    "input_dim": indim,
    "batch_size": 8,
    "filters_cnn": [8, 6],
    "units_lstm": [8,6],
    "kernel_size": [4,4],
    "hidden_act": ["relu", "relu"],
    "w_size": 30,
    "n_pred": 10,
     "n_clients":3,
     "n_rounds":3,
     }



modelDeep = flexanomalies.FlexAnomalyDetection(**kwargs)


In [26]:
processor =   federated_time_series_utils.TimeSeriesProcessor(window_size= kwargs["w_size"], step_size=4, future_prediction=True, n_pred=kwargs["n_pred"])
X_train_windows, y_train_windows, X_test_windows, y_test_windows, l_test_windows = processor.process_train_test(X_train, y_train, X_test, y_test,l_test=y_test)
print("X_train shape:", X_train_windows.shape)
print("y_train shape:", y_train_windows.shape)
print("X_test shape:", X_test_windows.shape)
print("y_test shape:", y_test_windows.shape)
print("l_test shape:",l_test_windows.shape)

X_train shape: (54421, 30, 38)
y_train shape: (54421, 10, 38)
X_test shape: (23318, 30, 38)
y_test shape: (23318, 10, 38)
l_test shape: (23318, 10)


In [27]:
modelDeep.fit(X_train_windows,y_train_windows)



Running round: 0

Training model at client.
Epoch 1/3
1633/1633 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - loss: 1.0356 - val_loss: 0.8646
Epoch 2/3
1633/1633 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - loss: 0.9652 - val_loss: 0.8645
Epoch 3/3
1633/1633 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - loss: 0.9584 - val_loss: 0.8646
Training model at client.
Epoch 1/3
1633/1633 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - loss: 0.8553 - val_loss: 1.0013
Epoch 2/3
1633/1633 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - loss: 0.8074 - val_loss: 1.0013
Epoch 3/3
1633/1633 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - loss: 0.8438 - val_loss: 1.0018
Training model at client.
Epoch 1/3
1633/1633 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - loss: 0.8833 - val_loss: 0.7260
Epoch 2/3
1633/1633 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - loss: 1.2212 - val_loss: 0.7260
Epoch 3/3
1633/1633 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - loss: 0.9043 - val_loss: 0.7260

Running round: 1

Training model at client.
Epoch 1/3
1633/1633 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - loss: 1.0955 - val_l

In [41]:
modelDeep.model.d_scores_ 
modelDeep.model.labels_


array([1, 0, 1, ..., 0, 1, 0])

In [30]:
modelDeep.evaluate(X_test_windows, y_test_windows, label_test = l_test_windows.flatten().astype("int"))

729/729 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
Acc: 80.145% 

Precision: 0.973 

F1score: 0.863 

Recall: 0.775 

AUC_ROC: 0.843 

